# 🚀 Google Colab Setup

**Before running this notebook on Colab:**
1. **Enable GPU**: Runtime → Change runtime type → Hardware accelerator → GPU (T4 or better)
2. **Mount Google Drive**: Run the cell below to access your data files
3. **Upload your data**: Make sure `cleaned_tweets.csv` is in your Drive at the specified path

**⚠️ IMPORTANT: BERT requires GPU for reasonable training time!**
- With GPU (T4): ~30-60 minutes total training
- Without GPU: 6-12 hours total training

In [ ]:
# Check if running on Colab
import os
IN_COLAB = 'COLAB_GPU' in os.environ

if IN_COLAB:
    print("✓ Running on Google Colab")
    
    # Mount Google Drive to access your data
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Set base path - ADJUST THIS to match your Drive folder structure
    BASE_PATH = '/content/drive/MyDrive/Sentiment140'
    
    # Check GPU availability - CRITICAL FOR BERT!
    import torch
    if torch.cuda.is_available():
        print(f"✓ GPU Available: {torch.cuda.get_device_name(0)}")
        print(f"✓ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
        print("✓ BERT training will be fast!")
    else:
        print("⚠️ WARNING: No GPU detected! BERT training will be VERY SLOW.")
        print("   Please enable GPU: Runtime → Change runtime type → Hardware accelerator → GPU")
    
    # Install required packages
    print("\n📦 Installing required packages...")
    !pip install -q transformers torch
    
    # Create necessary directories in Drive if they don't exist
    os.makedirs(os.path.join(BASE_PATH, 'models', 'dl', 'bert_model'), exist_ok=True)
    os.makedirs(os.path.join(BASE_PATH, 'visuals', 'charts'), exist_ok=True)
    os.makedirs(os.path.join(BASE_PATH, 'visuals', 'confusion_matrices'), exist_ok=True)
    os.makedirs(os.path.join(BASE_PATH, 'reports'), exist_ok=True)
    
    print("✓ All directories ready!")
else:
    print("Running locally")
    BASE_PATH = '..'

# BERT Fine-Tuning for Sentiment Analysis

This notebook implements transfer learning using pre-trained BERT (Bidirectional Encoder Representations from Transformers) for sentiment classification on the Sentiment140 dataset.

## 1. Import Libraries and Setup

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datetime
import warnings
warnings.filterwarnings('ignore')

# Transformers and PyTorch
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, RandomSampler, SequentialSampler
from torch.optim import AdamW
from transformers import (
    BertTokenizer, BertForSequenceClassification,
    get_linear_schedule_with_warmup
)

# Sklearn utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, 
    accuracy_score, f1_score, precision_score, recall_score
)

# Add src to path (only for local, not needed in Colab)
if 'BASE_PATH' not in dir():
    BASE_PATH = '..'
sys.path.append(os.path.join(BASE_PATH, 'src'))

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Check device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory Available: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
print("\nSetup complete!")

## 2. Load and Prepare Data

In [ ]:
# Load cleaned data
data_path = os.path.join(BASE_PATH, 'data', 'processed', 'cleaned_tweets.csv')
df = pd.read_csv(data_path)

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nClass distribution:")
print(df['sentiment'].value_counts())
print(f"\nSample tweets:")
df.head()

In [ ]:
# For BERT, we'll use a smaller sample for faster training
# You can increase this if you have sufficient GPU memory
SAMPLE_SIZE = 50000  # Adjust based on your hardware capabilities

# Sample data if dataset is large
if len(df) > SAMPLE_SIZE:
    df_sample = df.sample(n=SAMPLE_SIZE, random_state=42)
    print(f"\nUsing sample of {SAMPLE_SIZE} tweets for faster training")
else:
    df_sample = df
    print(f"\nUsing full dataset of {len(df)} tweets")

# Prepare texts and labels
texts = df_sample['text_clean'].values
labels = df_sample['sentiment'].values

print(f"\nFinal dataset size: {len(texts)}")
print(f"Label distribution: {np.bincount(labels)}")

## 3. Load BERT Tokenizer and Tokenize Data

In [ ]:
# Load pre-trained BERT tokenizer
MODEL_NAME = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)

print(f"Loaded tokenizer: {MODEL_NAME}")
print(f"Vocabulary size: {tokenizer.vocab_size}")
print(f"\nTokenizer special tokens:")
print(f"  PAD token: {tokenizer.pad_token} (ID: {tokenizer.pad_token_id})")
print(f"  CLS token: {tokenizer.cls_token} (ID: {tokenizer.cls_token_id})")
print(f"  SEP token: {tokenizer.sep_token} (ID: {tokenizer.sep_token_id})")

In [ ]:
# Tokenization parameters
MAX_LENGTH = 128  # Maximum sequence length for BERT

# Tokenize all texts
print("Tokenizing texts...")
encoded_data = tokenizer.batch_encode_plus(
    texts.tolist(),
    add_special_tokens=True,        # Add [CLS] and [SEP] tokens
    max_length=MAX_LENGTH,           # Max length to pad/truncate
    padding='max_length',            # Pad to max_length
    truncation=True,                 # Truncate longer sequences
    return_attention_mask=True,      # Return attention masks
    return_tensors='pt'              # Return PyTorch tensors
)

input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']
labels_tensor = torch.tensor(labels)

print(f"\nTokenization complete!")
print(f"Input IDs shape: {input_ids.shape}")
print(f"Attention masks shape: {attention_masks.shape}")
print(f"Labels shape: {labels_tensor.shape}")
print(f"\nExample tokenized sequence:")
print(f"Text: {texts[0][:100]}...")
print(f"Token IDs: {input_ids[0][:20]}")  # First 20 tokens

## 4. Create Train/Validation/Test DataLoaders

In [ ]:
# Split data into train, validation, and test sets
from sklearn.model_selection import train_test_split

# First split: separate test set (20%)
train_val_inputs, test_inputs, train_val_labels, test_labels, train_val_masks, test_masks = train_test_split(
    input_ids, labels_tensor, attention_masks, test_size=0.2, random_state=42, stratify=labels_tensor
)

# Second split: separate validation set from training (10% of original, ~12.5% of remaining)
train_inputs, val_inputs, train_labels, val_labels, train_masks, val_masks = train_test_split(
    train_val_inputs, train_val_labels, train_val_masks, test_size=0.125, random_state=42, stratify=train_val_labels
)

print(f"Training samples: {len(train_inputs)}")
print(f"Validation samples: {len(val_inputs)}")
print(f"Test samples: {len(test_inputs)}")

In [ ]:
# Create DataLoaders
BATCH_SIZE = 16  # Adjust based on GPU memory

# Create TensorDatasets
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)
test_dataset = TensorDataset(test_inputs, test_masks, test_labels)

# Create DataLoaders
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=BATCH_SIZE
)

val_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=BATCH_SIZE
)

test_dataloader = DataLoader(
    test_dataset,
    sampler=SequentialSampler(test_dataset),
    batch_size=BATCH_SIZE
)

print(f"DataLoaders created!")
print(f"  Training batches: {len(train_dataloader)}")
print(f"  Validation batches: {len(val_dataloader)}")
print(f"  Test batches: {len(test_dataloader)}")

## 5. Load Pre-trained BERT Model

In [ ]:
# Load pre-trained BERT model with sequence classification head
model = BertForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=2,           # Binary classification
    output_attentions=False,
    output_hidden_states=False
)

# Move model to GPU if available
model.to(device)

print(f"Loaded model: {MODEL_NAME}")
print(f"Number of parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")
print(f"\nModel architecture:")
print(model)

## 6. Setup Optimizer and Learning Rate Scheduler

In [ ]:
# Training hyperparameters
EPOCHS = 3  # Typical for BERT fine-tuning
LEARNING_RATE = 2e-5  # Recommended for BERT
WARMUP_STEPS = 0
EPSILON = 1e-8

# Create optimizer
optimizer = AdamW(
    model.parameters(),
    lr=LEARNING_RATE,
    eps=EPSILON
)

# Calculate total training steps
total_steps = len(train_dataloader) * EPOCHS

# Create learning rate scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=WARMUP_STEPS,
    num_training_steps=total_steps
)

print(f"Training configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Total training steps: {total_steps}")
print(f"  Warmup steps: {WARMUP_STEPS}")

## 7. Training Loop

In [ ]:
import time

def format_time(elapsed):
    """Format time in hh:mm:ss"""
    elapsed_rounded = int(round(elapsed))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# Training history
training_stats = []

print("Starting training...\n")
total_t0 = time.time()

for epoch in range(EPOCHS):
    print(f"{'='*70}")
    print(f"Epoch {epoch + 1}/{EPOCHS}")
    print(f"{'='*70}")
    
    # Training phase
    t0 = time.time()
    total_train_loss = 0
    model.train()
    
    for step, batch in enumerate(train_dataloader):
        # Progress update every 100 batches
        if step % 100 == 0 and step != 0:
            elapsed = format_time(time.time() - t0)
            print(f"  Batch {step:>5,} of {len(train_dataloader):>5,}.    Elapsed: {elapsed}.")
        
        # Unpack batch and move to GPU
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Clear gradients
        model.zero_grad()
        
        # Forward pass
        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )
        
        loss = outputs.loss
        total_train_loss += loss.item()
        
        # Backward pass
        loss.backward()
        
        # Clip gradients to prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # Update parameters
        optimizer.step()
        scheduler.step()
    
    # Calculate average training loss
    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)
    
    print(f"\n  Average training loss: {avg_train_loss:.4f}")
    print(f"  Training epoch took: {training_time}")
    
    # Validation phase
    print(f"\nRunning Validation...")
    t0 = time.time()
    
    model.eval()
    total_eval_accuracy = 0
    total_eval_loss = 0
    
    for batch in val_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():
            outputs = model(
                b_input_ids,
                attention_mask=b_input_mask,
                labels=b_labels
            )
        
        loss = outputs.loss
        logits = outputs.logits
        
        total_eval_loss += loss.item()
        
        # Calculate accuracy
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        pred_flat = np.argmax(logits, axis=1).flatten()
        labels_flat = label_ids.flatten()
        total_eval_accuracy += np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)
    avg_val_loss = total_eval_loss / len(val_dataloader)
    validation_time = format_time(time.time() - t0)
    
    print(f"  Validation Loss: {avg_val_loss:.4f}")
    print(f"  Validation Accuracy: {avg_val_accuracy:.4f}")
    print(f"  Validation took: {validation_time}")
    
    # Record stats
    training_stats.append({
        'epoch': epoch + 1,
        'Training Loss': avg_train_loss,
        'Valid. Loss': avg_val_loss,
        'Valid. Accur.': avg_val_accuracy,
        'Training Time': training_time,
        'Validation Time': validation_time
    })

print(f"\n{'='*70}")
print(f"Training complete!")
print(f"Total training took {format_time(time.time() - total_t0)}")

In [ ]:
# Display training statistics
training_stats_df = pd.DataFrame(training_stats)
print("\nTraining Statistics:")
print(training_stats_df)

## 8. Visualize Training Progress

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot training and validation loss
axes[0].plot(training_stats_df['epoch'], training_stats_df['Training Loss'], 
             marker='o', label='Training Loss', linewidth=2)
axes[0].plot(training_stats_df['epoch'], training_stats_df['Valid. Loss'], 
             marker='s', label='Validation Loss', linewidth=2)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot validation accuracy
axes[1].plot(training_stats_df['epoch'], training_stats_df['Valid. Accur.'], 
             marker='o', color='green', linewidth=2)
axes[1].set_title('Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0.7, 1.0])

plt.tight_layout()
plt.savefig(os.path.join(BASE_PATH, 'visuals', 'charts', 'bert_training_history.png'), dpi=300, bbox_inches='tight')
plt.show()

## 9. Evaluate on Test Set

In [ ]:
print("Evaluating on test set...")

model.eval()

predictions = []
true_labels = []

for batch in test_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    
    with torch.no_grad():
        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask
        )
    
    logits = outputs.logits
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    predictions.extend(np.argmax(logits, axis=1).flatten())
    true_labels.extend(label_ids.flatten())

predictions = np.array(predictions)
true_labels = np.array(true_labels)

print("\nBERT Model Evaluation on Test Set:")
print("="*50)
print(f"Accuracy: {accuracy_score(true_labels, predictions):.4f}")
print(f"Precision: {precision_score(true_labels, predictions):.4f}")
print(f"Recall: {recall_score(true_labels, predictions):.4f}")
print(f"F1-Score: {f1_score(true_labels, predictions):.4f}")
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=['Negative', 'Positive']))

## 10. Confusion Matrix

In [ ]:
# Plot confusion matrix
cm = confusion_matrix(true_labels, predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negative', 'Positive'],
            yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix - BERT Model', fontsize=14, fontweight='bold')
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(BASE_PATH, 'visuals', 'confusion_matrices', 'bert_confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

## 11. Save BERT Model

In [ ]:
# Create directory if it doesn't exist
output_dir = os.path.join(BASE_PATH, 'models', 'dl', 'bert_model')
os.makedirs(output_dir, exist_ok=True)

print(f"Saving model to {output_dir}")

# Save model and tokenizer
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("\nBERT model and tokenizer saved successfully!")
print(f"\nTo load the model later:")
print(f"  from transformers import BertForSequenceClassification, BertTokenizer")
print(f"  model = BertForSequenceClassification.from_pretrained('{output_dir}')")
print(f"  tokenizer = BertTokenizer.from_pretrained('{output_dir}')")

## 12. Test Predictions on Sample Texts

In [ ]:
def predict_sentiment(text, model, tokenizer, device):
    """
    Predict sentiment for a given text
    """
    model.eval()
    
    # Tokenize
    encoded = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    
    # Predict
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()
        probabilities = torch.softmax(logits, dim=1)[0]
    
    sentiment = "Positive" if prediction == 1 else "Negative"
    confidence = probabilities[prediction].item()
    
    return sentiment, confidence

# Test on sample texts
test_samples = [
    "I absolutely love this product! It's amazing!",
    "This is the worst experience ever. Very disappointed.",
    "The movie was okay, nothing special.",
    "Fantastic service and great quality!",
    "I hate waiting in long lines.",
    "Best day of my life!",
    "Terrible customer support, never buying again."
]

print("Sample Predictions:")
print("="*70)
for text in test_samples:
    sentiment, confidence = predict_sentiment(text, model, tokenizer, device)
    print(f"\nText: {text}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.4f})")

## 13. Summary and Conclusions

### Key Findings:

1. **Model Architecture:**
   - Pre-trained BERT (bert-base-uncased) with 110M parameters
   - Fine-tuned for binary sentiment classification
   - Used transfer learning to leverage pre-trained knowledge

2. **Training Configuration:**
   - 3 epochs (typical for BERT fine-tuning)
   - Learning rate: 2e-5 (recommended for BERT)
   - Batch size: 16
   - Max sequence length: 128 tokens

3. **Performance:**
   - Check the evaluation metrics above
   - BERT typically achieves higher accuracy than LSTM/GRU models
   - Benefits from contextual embeddings and bidirectional attention

4. **Advantages of BERT:**
   - Pre-trained on massive text corpus
   - Understands context better than sequential models
   - Captures semantic relationships effectively
   - State-of-the-art performance on NLP tasks

5. **Considerations:**
   - Requires more computational resources (GPU recommended)
   - Slower training compared to LSTM/GRU
   - Larger model size
   - Trade-off between performance and efficiency

### Next Steps:
- Compare BERT performance with ML models (notebook 02) and LSTM/GRU (notebook 03)
- Experiment with different BERT variants (RoBERTa, DistilBERT, etc.)
- Try ensemble methods combining multiple models
- Perform error analysis to understand misclassifications